In [27]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
import gseapy as gp
from singleR import annotate_with_singler

In [28]:
import torch, numpy as np, random
seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [29]:
import scanpy as sc

ST_full = sc.read_h5ad('/data/keerthana/integration/scMODAL/xenium_breast_cancer.h5ad')
ST_full.var_names_make_unique()
SC_full = sc.read_h5ad('/data/keerthana/integration/scMODAL/xenium_sc_data.h5ad')
SC_full.var_names_make_unique()

#subset
N_A = 2000
N_B = 2000

# Set a seed for reproducibility
rng = np.random.default_rng(seed=42)

A_idx = rng.choice(ST_full.n_obs, size=N_A, replace=False)
B_idx = rng.choice(SC_full.n_obs, size=N_B, replace=False)

ST = ST_full[A_idx].copy()
SC = SC_full[B_idx].copy()

In [30]:
import pandas as pd

# Define mapping from fine-grained ST types to major SC types
mapping = {
    # Lymphoid
    'B_Cells': 'B-cells',
    'CD4+_T_Cells': 'T-cells',
    'CD8+_T_Cells': 'T-cells',
    'Stromal_&_T_Cell_Hybrid': 'T-cells',
    'T_Cell_&_Tumor_Hybrid': 'T-cells',

    # Myeloid
    'Macrophages_1': 'Myeloid',
    'Macrophages_2': 'Myeloid',
    'Mast_Cells': 'Myeloid',
    'IRF7+_DCs': 'Myeloid',
    'LAMP3+_DCs': 'Myeloid',
    'TFR+_DCs': 'Myeloid',

    # Epithelial
    'Invasive_Tumor': 'Cancer Epithelial',
    'Prolif_Invasive_Tumor': 'Cancer Epithelial',
    'DCIS_1': 'Cancer Epithelial',
    'DCIS_2': 'Cancer Epithelial',
    'Myoepi_ACTA2+': 'Normal Epithelial',
    'Myoepi_KRT15+': 'Normal Epithelial',

    # Endothelial
    'Endothelial': 'Endothelial',
    'Perivascular-Like': 'Endothelial',

    # Stromal
    'Stromal': 'CAFs',

    # Unlabeled / others
    'Unlabeled': 'Unlabeled'
}

# Apply mapping
ST.obs['celltype_major'] = ST.obs['cell_type'].map(mapping)

# If any fine types weren't mapped
unmapped = ST.obs.loc[ST.obs['celltype_major'].isna(), 'cell_type'].unique()
if len(unmapped) > 0:
    print("Unmapped cell types:", unmapped)
else:
    print("✅ All cell types mapped successfully!")



✅ All cell types mapped successfully!


if "counts" not in ST.layers:
    ST.layers["counts"] = ST.X.copy()
# make sure sample is a category
ST.obs["sample"] = ST.obs.get("sample", "sample_1")
ST.obs["sample"] = ST.obs["sample"].astype(str).astype("category")


In [31]:
# Reorder ST and SC so that common columns are first and in the same order
shared_genes = [g for g in ST.var_names if g in SC.var_names]  # preserves ST order
ST = ST[:, shared_genes]
SC = SC[:, shared_genes]

# keep track of common_vars length
common_vars = shared_genes
shared_gene_num = len(common_vars)



In [32]:
ST.shape, SC.shape

((2000, 308), (2000, 308))

In [33]:
import scmodal
model = scmodal.model.Model(training_steps=1000)
model.preprocess(ST,SC, shared_gene_num= shared_gene_num,alpha = None, eta = None)

preprocess called with alpha: None and eta: None
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [34]:
model.train()

Begining time:  Tue Nov 25 01:39:44 2025
step 0, loss_D=0.053221, loss_GAN=-0.046050, loss_AE=359.125610, loss_Geo=-14.648248, loss_LA=2174.932617, loss_MNN=53.552708
Ending time:  Tue Nov 25 01:40:29 2025
Training takes 45.53 seconds


In [35]:
model.eval()

Begining time:  Tue Nov 25 01:40:29 2025
Ending time:  Tue Nov 25 01:40:29 2025
Evaluating takes 0.05 seconds


In [36]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

def impute_cell_types(model, ST, SC, n_neighbors=10):
    emb_A = model.latent[:ST.shape[0], :]
    emb_B = model.latent[ST.shape[0]:, :]

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(emb_B)
    knn_idx = nbrs.kneighbors(emb_A, return_distance=False)

    # Factorize string labels to numeric
    labels = np.array(SC.obs["celltype_major"].values)
    codes, uniques = pd.factorize(labels)
    neighbor_codes = codes[knn_idx]

    # Majority vote using bincount (fast + numeric)
    majority_idx = np.apply_along_axis(lambda x: np.bincount(x).argmax(), 1, neighbor_codes)
    majority_labels = uniques[majority_idx]

    ST.obs["imputed_cell_type"] = majority_labels
    return ST


In [37]:
ST = annotate_with_singler(
    model=model,  # optional
    ST=ST, 
    SC=SC, 
    cell_type_column="celltype_major",
    fine_tune=True,
    use_embeddings=True  # Set True to use model.latent like KNN
)
ari = adjusted_rand_score(ST.obs['celltype_major'], ST.obs['singler_cell_type'])
ari

SingleR Cell Type Annotation
Using latent embeddings for correlation...

Step 1: Computing Spearman correlations...
Correlation matrix shape: (2000, 2000)

Step 2: Aggregating scores per cell type...
Initial label distribution:
  Cancer Epithelial: 578
  CAFs: 330
  T-cells: 293
  Normal Epithelial: 290
  Myeloid: 180
  Endothelial: 100
  B-cells: 92
  PVL: 86
  Plasmablasts: 51

Step 3: Fine-tuning (threshold=0.05)...
Fine-tuning 2000 cells...
Final label distribution:
  Cancer Epithelial: 578
  CAFs: 330
  T-cells: 293
  Normal Epithelial: 290
  Myeloid: 180
  Endothelial: 100
  B-cells: 92
  PVL: 86
  Plasmablasts: 51

Annotation complete!


0.5483308103311969

In [38]:
ST = impute_cell_types(model, ST, SC, n_neighbors=10)
print(len(np.unique(ST.obs['imputed_cell_type'])))
from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(ST.obs['celltype_major'], ST.obs['imputed_cell_type'])
ari

9


0.6730675916378895

In [39]:
#actual loop starts here - now that we have first guess of cell types

In [40]:
# --- ensure counts layer is present and sample is categorical string ---
if "counts" not in ST.layers:
    ST.layers["counts"] = ST.X.copy()
# make sure sample is a category
ST.obs["sample"] = ST.obs.get("sample", "sample_1")
ST.obs["sample"] = ST.obs["sample"].astype(str).astype("category")

In [41]:
import scvi
import scanpy as sc
from scvi.external import SCVIVA
import logging

scvi.settings.seed = 42
scvi.settings.verbosity = logging.CRITICAL

scvi.model.SCVI.setup_anndata(
    ST,
    layer="counts",            # raw counts for modeling
    batch_key="sample",        # if you have one batch, set to a constant
    labels_key="imputed_cell_type",    # for SCANVI later
)

# 4. Train SCVI
vae = scvi.model.SCVI(ST)
vae.train(max_epochs=200)

scanvi_model = scvi.model.SCANVI.from_scvi_model(vae, unlabeled_category="Unknown")
scanvi_model.train(max_epochs=100)

ST.obsm["X_scANVI"] = scanvi_model.get_latent_representation()

setup_kwargs = {
        # "sample_key": "sample",  # column in adata.obs that contains the individual slide ID
        "labels_key": "imputed_cell_type",  # column in adata.obs that contains the cell type labels
        "cell_coordinates_key": "spatial",  # spatial coordinates key in adata.obsm
        "expression_embedding_key": "X_scANVI",  # expression embedding key in adata.obsm
        "sample_key": "sample"
    }
scvi.external.SCVIVA.preprocessing_anndata(
    ST,
    k_nn=20,  # number of nearest neighbors for spatial graph construction
    **setup_kwargs,
)
scvi.external.SCVIVA.setup_anndata(
    ST,
    layer="counts",  # adata layer that contains the raw counts
    batch_key="sample",  # column in adata.obs that contains the batch covariate
    **setup_kwargs,
)

nichevae = scvi.external.SCVIVA(ST, gene_likelihood="zinb")

Seed set to 42
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Training:   0%|          | 0/200 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Training:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


Saved niche_indexes and niche_distances in adata.obsm

Saved niche_composition in adata.obsm

Saved niche_activation in adata.obsm

In [42]:
n_iter = 9

In [43]:
ckpt = "scviva_ckpt"

aris = []
for i in range(n_iter):
    print(f"🔄 Iteration {i+1}/{n_iter}")

    # 1. INIT (1st iteration) or LOAD (later iterations)
    if i == 0:
        scvi.external.SCVIVA.setup_anndata(
            ST,
            layer="counts",
            batch_key="sample",
            **setup_kwargs,
        )
        nichevae = scvi.external.SCVIVA(ST, gene_likelihood="zinb")
    else:
        # Load SCVIVA from checkpoint
        nichevae = scvi.external.SCVIVA.load(ckpt, ST)


    # 2. TRAIN SCVIVA
    nichevae.train(
        max_epochs=50,
        early_stopping=True,
        batch_size=min(512, ST.shape[0]),
        plan_kwargs={
            "lr": 5e-4,
        },
    )

    # 3. SAVE CHECKPOINT
    nichevae.save(ckpt, overwrite=True)

    # 4. UPDATE LATENT ("η")
    ST.obsm["X_scVIVA"] = nichevae.get_latent_representation()
    ST.obsm["X_scVIVA"] = pd.DataFrame(ST.obsm["X_scVIVA"], index=ST.obs.index)

    # 5. UPDATE NIHCHEVAE ("α", "η") INTO YOUR MODEL
    alpha = ST.obsm['niche_composition']
    eta = ST.obsm["X_scVIVA"]
    model.update_embeddings(alpha=alpha, eta=eta)

    # 6. TRAIN DOWNSTREAM MODEL (INITIALIZE ONLY ON FIRST ITERATION)
    if i == 0:
        model.train(load_ckpt=False)
    else:
        model.train(load_ckpt=True)
    model.eval()

    # 7. IMPUTE CELL TYPES
    ST = impute_cell_types(model, ST, SC, n_neighbors=10)

    # 8. Evaluate ARI
    ari = adjusted_rand_score(ST.obs['celltype_major'], ST.obs['imputed_cell_type'])
    aris.append(ari)
    print(f"Iteration ARI: {ari}")

    prev_ari = ari


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


🔄 Iteration 1/9


Training:   0%|          | 0/50 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


✅ Updated alpha and eta embeddings in the model (weights preserved).
Begining time:  Tue Nov 25 02:33:04 2025
step 0, loss_D=0.192580, loss_GAN=-0.109319, loss_AE=182.745071, loss_Geo=-16.196329, loss_LA=1562.466919, loss_MNN=26.988274
Ending time:  Tue Nov 25 02:33:53 2025
Training takes 48.65 seconds
Begining time:  Tue Nov 25 02:33:53 2025
Ending time:  Tue Nov 25 02:33:53 2025
Evaluating takes 0.06 seconds


...

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Iteration ARI: 0.6083055116159314
🔄 Iteration 9/9


Training:   0%|          | 0/50 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


Monitored metric elbo_validation did not improve in the last 45 records. Best score: 253.042. Signaling Trainer to stop.
✅ Updated alpha and eta embeddings in the model (weights preserved).
Begining time:  Tue Nov 25 02:40:16 2025
🧩 Loading checkpoint with shape-safe mapping...
⚠️ Skipping optimizer_G load due to mismatch: local variable 'optimizer_G' referenced before assignment
⚠️ Skipping optimizer_D load due to mismatch: local variable 'optimizer_D' referenced before assignment
step 0, loss_D=0.804664, loss_GAN=-0.773244, loss_AE=3.611010, loss_Geo=-19.196201, loss_LA=0.092727, loss_MNN=0.983395
Ending time:  Tue Nov 25 02:41:06 2025
Training takes 50.20 seconds
Begining time:  Tue Nov 25 02:41:07 2025
Ending time:  Tue Nov 25 02:41:07 2025
Evaluating takes 0.06 seconds
Iteration ARI: 0.6075058843105359


In [44]:
np.unique(nichevae.adata.obs['imputed_cell_type'])

array(['B-cells', 'CAFs', 'Cancer Epithelial', 'Endothelial', 'Myeloid',
       'Normal Epithelial', 'PVL', 'Plasmablasts', 'T-cells'],
      dtype=object)

In [45]:
aris

[0.6383448292998757,
 0.6076924916981992,
 0.6138890561504543,
 0.6278733386496866,
 0.617843194328631,
 0.6223115833976512,
 0.6244447979950275,
 0.6083055116159314,
 0.6075058843105359]

In [46]:
ST = annotate_with_singler(
    model=model,  # optional
    ST=ST, 
    SC=SC, 
    cell_type_column="celltype_major",
    fine_tune=True,
    use_embeddings=True  # Set True to use model.latent like KNN
)
ari = adjusted_rand_score(ST.obs['celltype_major'], ST.obs['singler_cell_type'])
ari

SingleR Cell Type Annotation
Using latent embeddings for correlation...

Step 1: Computing Spearman correlations...
Correlation matrix shape: (2000, 2000)

Step 2: Aggregating scores per cell type...
Initial label distribution:
  Cancer Epithelial: 574
  T-cells: 361
  CAFs: 287
  Normal Epithelial: 261
  Myeloid: 172
  Endothelial: 116
  PVL: 97
  Plasmablasts: 71
  B-cells: 61

Step 3: Fine-tuning (threshold=0.05)...
Fine-tuning 2000 cells...
Final label distribution:
  Cancer Epithelial: 574
  T-cells: 361
  CAFs: 287
  Normal Epithelial: 261
  Myeloid: 172
  Endothelial: 116
  PVL: 97
  Plasmablasts: 71
  B-cells: 61

Annotation complete!


0.5023372883184692

In [48]:
from sklearn.metrics import classification_report

report = classification_report(
    ST.obs['celltype_major'],
    ST.obs['singler_cell_type']
)

print(report)

                   precision    recall  f1-score   support

          B-cells       0.25      0.24      0.24        62
             CAFs       0.98      0.57      0.72       496
Cancer Epithelial       0.92      0.70      0.80       752
      Endothelial       0.89      0.83      0.86       124
          Myeloid       0.77      0.75      0.76       175
Normal Epithelial       0.27      0.56      0.36       125
              PVL       0.00      0.00      0.00         0
     Plasmablasts       0.00      0.00      0.00         0
          T-cells       0.48      0.97      0.64       177
        Unlabeled       0.00      0.00      0.00        89

         accuracy                           0.65      2000
        macro avg       0.45      0.46      0.44      2000
     weighted avg       0.78      0.65      0.69      2000



In [49]:
from sklearn.metrics import classification_report

report = classification_report(
    ST.obs['celltype_major'],
    ST.obs['imputed_cell_type']
)

print(report)

                   precision    recall  f1-score   support

          B-cells       0.07      0.05      0.06        62
             CAFs       0.99      0.50      0.66       496
Cancer Epithelial       0.93      0.90      0.92       752
      Endothelial       0.91      0.77      0.83       124
          Myeloid       0.73      0.74      0.74       175
Normal Epithelial       0.63      0.63      0.63       125
              PVL       0.00      0.00      0.00         0
     Plasmablasts       0.00      0.00      0.00         0
          T-cells       0.40      0.99      0.57       177
        Unlabeled       0.00      0.00      0.00        89

         accuracy                           0.70      2000
        macro avg       0.47      0.46      0.44      2000
     weighted avg       0.79      0.70      0.72      2000

